In [0]:
import requests

In [0]:
url = 'https://raw.githubusercontent.com/enuganti/data-engineer/refs/heads/main/PySparkScenario/Scenario%2002%20%20How%20to%20find%20new%20records%20by%20compare%20target%20table%20%26%20csv/initload.csv'

In [0]:
def getval:
    res = requests.get(url)
    return res
df = spark.read.csv(getval(), header=True)
df.write.mode("overwrite").saveAsTable("source")

In [0]:
import requests

def getval(url):
    res = requests.get(url)
    with open('/tmp/tempfile.csv', 'wb') as f:
        f.write(res.content)
    return '/tmp/tempfile.csv'

url = 'your_url_here'
df = spark.read.csv(getval(url), header=True)
df.write.mode("overwrite").saveAsTable("source")

In [0]:
import requests

url = 'https://api.fda.gov/drug/drugsfda.json?limit=100'
response = requests.get(url)

df_drug = spark.createDataFrame(response.json()['results'])
display(df_drug)

In [0]:
import requests
from pyspark.sql.types import StructType, StructField, StringType, ArrayType

url = 'https://api.fda.gov/drug/drugsfda.json?limit=100'
response = requests.get(url)

# Define the schema
schema = StructType([
    StructField('products', ArrayType(StructType([
        StructField('product_number', StringType(), True),
        StructField('reference_drug', StringType(), True),
        StructField('brand_name', StringType(), True),
        StructField('active_ingredients', ArrayType(StructType([
            StructField('name', StringType(), True),
            StructField('strength', StringType(), True)
        ])), True),
        StructField('reference_standard', StringType(), True)
    ])), True),
    StructField('submission_type', StringType(), True),
    StructField('submission_status', StringType(), True),
    StructField('submission_status_date', StringType(), True),
    StructField('review_priority', StringType(), True),
    StructField('application_number', StringType(), True),
    StructField('sponsor_name', StringType(), True)
])

df_drug = spark.createDataFrame(response.json()['results'], schema=schema)
display(df_drug)

In [0]:
df_drug.write.mode("overwrite").saveAsTable("API_DRUG")

In [0]:
df_drug.select(df_drug["products"]).display()

In [0]:
df_drug.select(df_drug["products"][0]["brand_name"]).display()

In [0]:
df_drug.select('*',df_drug["products"][0]["brand_name"],df_drug["products"][0]["active_ingredients"][0]["name"].alias("active_name"),df_drug["products"][0]["active_ingredients"][0]["strength"].alias("strength"),df_drug["products"][0]["reference_standard"].alias("reference_standard")).show(truncate=False)

In [0]:
df_drug.select('*',df_drug["products"][0]["brand_name"],df_drug["products"][0]["active_ingredients"][0]["name"].alias("active_name"),df_drug["products"][0]["active_ingredients"][0]["strength"].alias("strength"),df_drug["products"][0]["reference_standard"].alias("reference_standard")).display()

In [0]:
df_drug.select(df_drug.products.product_number[0]).display()


In [0]:
df_drug.select(df_drug["products"][0]["product_number"].alias("product_number"),df_drug["products"][0]["reference_drug"].alias("reference_drug"),df_drug["products"][0]["brand_name"].alias("brand_name"),df_drug["products"][0]["active_ingredients"][0]["name"].alias("ingredients_name"),df_drug["products"][0]["active_ingredients"][0]["strength"].alias("strength"),df_drug["products"][0]["reference_standard"].alias("reference_standard")).display()


In [0]:
df_drug_api = df_drug.select('*',df_drug["products"][0]["product_number"].alias("product_number"),df_drug["products"][0]["reference_drug"].alias("reference_drug"),df_drug["products"][0]["brand_name"].alias("brand_name"),df_drug["products"][0]["active_ingredients"][0]["name"].alias("ingredients_name"),df_drug["products"][0]["active_ingredients"][0]["strength"].alias("strength"),df_drug["products"][0]["reference_standard"].alias("reference_standard")).drop("products").display()

df_drug_api.write.format("delta").mode("overwrite").save('/Volumes/workspace/default/emp/api_drug_new')

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

df_drug_api = df_drug.withColumn("submission_type", replace("submission_type", "", "api_drug")).withColumn("submission_status", replace("submission_status", "", "active")).withColumn("submission_status_date", replace("submission_status_date","",current_date())).withColumn("sponsor_name", replace("sponsor_name", "", "FDA"))\
    .select(df_drug["submission_type"],
    df_drug["submission_status"],
    df_drug["submission_status_date"],   
    df_drug["sponsor_name"],    
    df_drug["products"][0]["product_number"].alias("product_number"),
    df_drug["products"][0]["reference_drug"].alias("reference_drug"),
    df_drug["products"][0]["brand_name"].alias("brand_name"),
    df_drug["products"][0]["active_ingredients"][0]["name"].alias("ingredients_name"),
    df_drug["products"][0]["active_ingredients"][0]["strength"].alias("strength"),
    df_drug["products"][0]["reference_standard"].alias("reference_standard")
).drop("products")

display(df_drug_api)

df_drug_api.write.format("delta").mode("overwrite").save('/Volumes/workspace/default/emp/api_drug_new')

In [0]:
from pyspark.sql.functions import col, current_date, lit, when

df_drug_api = df_drug.withColumn(
    "submission_type", 
    when(col("submission_type") == "", "api_drug").otherwise(col("submission_type"))
).withColumn(
    "submission_status", 
    when(col("submission_status") == "", "active").otherwise(col("submission_status"))
).withColumn(
    "submission_status_date", 
    when(col("submission_status_date") == "", current_date()).otherwise(col("submission_status_date"))
).withColumn(
    "sponsor_name", 
    when(col("sponsor_name") == "", "FDA").otherwise(col("sponsor_name"))
).select(
    col("submission_type"),
    col("submission_status"),
    col("submission_status_date"),   
    col("sponsor_name"),    
    col("products")[0]["product_number"].alias("product_number"),
    col("products")[0]["reference_drug"].alias("reference_drug"),
    col("products")[0]["brand_name"].alias("brand_name"),
    col("products")[0]["active_ingredients"][0]["name"].alias("ingredients_name"),
    col("products")[0]["active_ingredients"][0]["strength"].alias("strength"),
    col("products")[0]["reference_standard"].alias("reference_standard")
).drop("products")

display(df_drug_api)

df_drug_api.write.format("delta").mode("overwrite").save('/Volumes/workspace/default/emp/api_drug_new')

In [0]:
df_dg = spark.read.format("delta").load("/Volumes/workspace/default/emp/api_drug_new")
df_dg.display()

In [0]:
df_dg_new = df_dg.na.fill("0").display()


df_dg_new = df_dg.na.fill("0", subset= ["submission_status_date"]).display()

df_dg_new = df_dg.na.replace([None],["api_type"], subset = ["submission_type"]).display()

In [0]:
%python
df_dg_new = df_dg.na.fill("0")
display(df_dg_new)

df_dg_new = df_dg.na.fill("0", subset=["submission_status_date"])
display(df_dg_new)

df_dg_new = df_dg.na.replace(None, "api_type", subset=["submission_type"])
display(df_dg_new)

In [0]:
%sql
select * from workspace.default.api_drug

In [0]:
import requests
url = 'https://api.fda.gov/drug/drugsfda.json?limit=100'
response = requests.get(url)
df_drugs = spark.createDataFrame(response.json()['results'],  schema = StructType([
    StructField('products', ArrayType(StructType([
        StructField('product_number', StringType(), True),
        StructField('reference_drug', StringType(), True),
        StructField('brand_name', StringType(), True),
        StructField('active_ingredients', ArrayType(StructType([
            StructField('name', StringType(), True),
            StructField('strength', StringType(), True)
        ])), True),
        StructField('reference_standard', StringType(), True)
    ])), True),
    StructField('submission_type', StringType(), True),
    StructField('submission_status', StringType(), True),
    StructField('submission_status_date', StringType(), True),
    StructField('review_priority', StringType(), True),
    StructField('application_number', StringType(), True),
    StructField('sponsor_name', StringType(), True)
]))
df_drugs.write.mode("overwrite").saveAsTable("API_DRUG")
df_drugs.display()

In [0]:
import numpy as np
import pandas as pd

# Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

# Generate a pandas DataFrame
pdf = pd.DataFrame(np.random.rand(100, 3))

# Create a Spark DataFrame from a pandas DataFrame using Arrow
df = spark.createDataFrame(pdf)

# Convert the Spark DataFrame back to a pandas DataFrame using Arrow
result_pdf = df.select("*").toPandas()

In [0]:
%python
import numpy as np
import pandas as pd

# Generate a pandas DataFrame
pdf = pd.DataFrame(np.random.rand(100, 3))

# Create a Spark DataFrame from a pandas DataFrame
df = spark.createDataFrame(pdf)

# Convert the Spark DataFrame back to a pandas DataFrame
result_pdf = df.select("*").toPandas()

display(result_pdf)